In [2]:
import datetime, warnings, scipy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

#import pandas
from keras.models import Sequential
from keras.layers import Dense # fully connected layers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
merged_flights = pd.read_csv('Data/merged_flights.csv', index_col=0)
merged_flights.head()

,SCHEDULED_DATE,DAY_OF_WEEK,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,...,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE,Count Flights,Enplanements,area
0,2015-01-01,4,AS,N407AS,ANC,SEA,00:05:00,23:54:00,21.0,00:15:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west
1,2015-01-01,4,AS,N309AS,ANC,SEA,00:45:00,00:41:00,17.0,00:58:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west
2,2015-01-01,4,DL,N3743H,ANC,SEA,00:45:00,00:31:00,25.0,00:56:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west
3,2015-01-01,4,AS,N413AS,ANC,PDX,00:50:00,00:46:00,11.0,00:57:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west
4,2015-01-01,4,US,N804AW,ANC,PHX,01:52:00,01:43:00,21.0,02:04:00,...,ANC,Ted Stevens Anchorage International Airport,Anchorage,AK,USA,61.17432,-149.99619,7888,"2,713,843",west


In [4]:
merged_flights.shape

(2818553, 26)

In [5]:
flight_times = pd.read_csv('Data/flight_times.csv', index_col=0)
flight_times.head()

,SCHEDULED_DATE,DAY_OF_WEEK,AIRLINE,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,DEPARTURE_DELAY,SPEED
id,,,,,,,,,,,,,,,,
0,2015-01-01,4,AS,N407AS,ANC,SEA,00:05:00,23:54:00,21.0,00:15:00,205.0,1448,04:30:00,-22.0,-11.0,423.805
1,2015-01-01,4,AA,N3KUAA,LAX,PBI,00:10:00,00:02:00,12.0,00:14:00,280.0,2330,07:50:00,-9.0,-8.0,499.286
2,2015-01-01,4,US,N171US,SFO,CLT,00:20:00,00:18:00,16.0,00:34:00,286.0,2296,08:06:00,5.0,-2.0,481.678
3,2015-01-01,4,AA,N3HYAA,LAX,MIA,00:20:00,00:15:00,15.0,00:30:00,285.0,2342,08:05:00,-9.0,-5.0,493.053
4,2015-01-01,4,AS,N527AS,SEA,ANC,00:25:00,00:24:00,11.0,00:35:00,235.0,1448,03:20:00,-21.0,-1.0,369.702


In [5]:
merged_flights.columns

Index(['SCHEDULED_DATE', 'DAY_OF_WEEK', 'AIRLINE', 'TAIL_NUMBER',
       'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE',
       'DEPARTURE_TIME', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME',
       'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'DEPARTURE_DELAY',
       'SPEED', 'IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE',
       'LONGITUDE', 'Count Flights', 'Enplanements', 'area'],
      dtype='object')

In [6]:
flight_times.columns

Index(['SCHEDULED_DATE', 'DAY_OF_WEEK', 'AIRLINE', 'TAIL_NUMBER',
       'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE',
       'DEPARTURE_TIME', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME',
       'DISTANCE', 'SCHEDULED_ARRIVAL', 'ARRIVAL_DELAY', 'DEPARTURE_DELAY',
       'SPEED'],
      dtype='object')

#### Drop unnecessary (?) columns, repeated info

In [6]:
merged_flights=merged_flights.drop(columns=['TAIL_NUMBER','SCHEDULED_DEPARTURE', 'DEPARTURE_TIME','AIRPORT','COUNTRY','IATA_CODE'])
merged_flights

,SCHEDULED_DATE,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,DEPARTURE_DELAY,SPEED,CITY,STATE,LATITUDE,LONGITUDE,Count Flights,Enplanements,area
0,2015-01-01,4,AS,ANC,SEA,21.0,00:15:00,205.0,1448,04:30:00,-22.0,-11.0,423.805,Anchorage,AK,61.17432,-149.99619,7888,"2,713,843",west
1,2015-01-01,4,AS,ANC,SEA,17.0,00:58:00,204.0,1448,05:09:00,-14.0,-4.0,425.882,Anchorage,AK,61.17432,-149.99619,7888,"2,713,843",west
2,2015-01-01,4,DL,ANC,SEA,25.0,00:56:00,210.0,1448,05:15:00,-24.0,-14.0,413.714,Anchorage,AK,61.17432,-149.99619,7888,"2,713,843",west
3,2015-01-01,4,AS,ANC,PDX,11.0,00:57:00,215.0,1542,05:25:00,-18.0,-4.0,430.326,Anchorage,AK,61.17432,-149.99619,7888,"2,713,843",west
4,2015-01-01,4,US,ANC,PHX,21.0,02:04:00,323.0,2552,09:15:00,-10.0,-9.0,474.056,Anchorage,AK,61.17432,-149.99619,7888,"2,713,843",west
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818548,2015-06-25,4,B6,HYA,JFK,9.0,13:24:00,72.0,196,14:21:00,-12.0,6.0,163.333,Hyannis,MA,41.66934,-70.28036,12,"31,027",south
2818549,2015-06-26,5,B6,HYA,JFK,5.0,13:20:00,72.0,196,14:21:00,-18.0,6.0,163.333,Hyannis,MA,41.66934,-70.28036,12,"31,027",south
2818550,2015-06-27,6,B6,HYA,JFK,16.0,13:24:00,72.0,196,14:21:00,-7.0,-1.0,163.333,Hyannis,MA,41.66934,-70.28036,12,"31,027",south
2818551,2015-06-29,1,B6,HYA,JFK,6.0,13:11:00,72.0,196,14:21:00,-22.0,-4.0,163.333,Hyannis,MA,41.66934,-70.28036,12,"31,027",south


In [7]:
LR_train =merged_flights.drop(columns=['SCHEDULED_DATE','DAY_OF_WEEK','WHEELS_OFF','SCHEDULED_ARRIVAL','LONGITUDE','LATITUDE','Enplanements'])
LR_train

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,TAXI_OUT,SCHEDULED_TIME,DISTANCE,ARRIVAL_DELAY,DEPARTURE_DELAY,SPEED,CITY,STATE,Count Flights,area
0,AS,ANC,SEA,21.0,205.0,1448,-22.0,-11.0,423.805,Anchorage,AK,7888,west
1,AS,ANC,SEA,17.0,204.0,1448,-14.0,-4.0,425.882,Anchorage,AK,7888,west
2,DL,ANC,SEA,25.0,210.0,1448,-24.0,-14.0,413.714,Anchorage,AK,7888,west
3,AS,ANC,PDX,11.0,215.0,1542,-18.0,-4.0,430.326,Anchorage,AK,7888,west
4,US,ANC,PHX,21.0,323.0,2552,-10.0,-9.0,474.056,Anchorage,AK,7888,west
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818548,B6,HYA,JFK,9.0,72.0,196,-12.0,6.0,163.333,Hyannis,MA,12,south
2818549,B6,HYA,JFK,5.0,72.0,196,-18.0,6.0,163.333,Hyannis,MA,12,south
2818550,B6,HYA,JFK,16.0,72.0,196,-7.0,-1.0,163.333,Hyannis,MA,12,south
2818551,B6,HYA,JFK,6.0,72.0,196,-22.0,-4.0,163.333,Hyannis,MA,12,south


### One-Hot encoding

In [8]:
LR_X = LR_train.drop(columns=['ARRIVAL_DELAY'])
LR_Y = LR_train["ARRIVAL_DELAY"]

In [9]:
LR_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2818553 entries, 0 to 2818552
Data columns (total 12 columns):
 #   Column               Dtype  
---  ------               -----  
 0   AIRLINE              object 
 1   ORIGIN_AIRPORT       object 
 2   DESTINATION_AIRPORT  object 
 3   TAXI_OUT             float64
 4   SCHEDULED_TIME       float64
 5   DISTANCE             int64  
 6   DEPARTURE_DELAY      float64
 7   SPEED                float64
 8   CITY                 object 
 9   STATE                object 
 10  Count Flights        int64  
 11  area                 object 
dtypes: float64(4), int64(2), object(6)
memory usage: 279.6+ MB


In [10]:
# Find object columns for one-hot encoding
df_categories = LR_X.select_dtypes(include=['object']).copy()
df_categories.head()

,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,CITY,STATE,area
0,AS,ANC,SEA,Anchorage,AK,west
1,AS,ANC,SEA,Anchorage,AK,west
2,DL,ANC,SEA,Anchorage,AK,west
3,AS,ANC,PDX,Anchorage,AK,west
4,US,ANC,PHX,Anchorage,AK,west


In [87]:
#df_categories = pd.get_dummies(df_categories, columns=['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','CITY','STATE','area'], prefix = ['Airline','Origin','Dest','city','state','area'])
#print(df_categories.head())

Probably need to drop some of the categorical features, (CITY, STATE, area ?)

If we encode all of those features, we will end up with over a thousand columns

In [11]:
X = LR_train.drop(columns=['ARRIVAL_DELAY'])
Y = LR_train["ARRIVAL_DELAY"]
X = pd.get_dummies(X, columns=['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','CITY','STATE','area'], prefix = ['Airline','Origin','Dest','city','state','area'])
X

,TAXI_OUT,SCHEDULED_TIME,DISTANCE,DEPARTURE_DELAY,SPEED,Count Flights,Airline_AA,Airline_AS,Airline_B6,Airline_DL,...,state_VT,state_WA,state_WI,state_WV,state_WY,area_islands,area_midwest,area_northeast,area_south,area_west
0,21.0,205.0,1448,-11.0,423.805,7888,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,17.0,204.0,1448,-4.0,425.882,7888,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,25.0,210.0,1448,-14.0,413.714,7888,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,11.0,215.0,1542,-4.0,430.326,7888,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,21.0,323.0,2552,-9.0,474.056,7888,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818548,9.0,72.0,196,6.0,163.333,12,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2818549,5.0,72.0,196,6.0,163.333,12,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2818550,16.0,72.0,196,-1.0,163.333,12,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2818551,6.0,72.0,196,-4.0,163.333,12,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


### Getting encoded features

In [12]:
LR_X = pd.get_dummies(LR_X, columns=['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','CITY','STATE','area'], prefix = ['Airline','Origin','Dest','city','state','area'])
LR_X

,TAXI_OUT,SCHEDULED_TIME,DISTANCE,DEPARTURE_DELAY,SPEED,Count Flights,Airline_AA,Airline_AS,Airline_B6,Airline_DL,...,state_VT,state_WA,state_WI,state_WV,state_WY,area_islands,area_midwest,area_northeast,area_south,area_west
0,21.0,205.0,1448,-11.0,423.805,7888,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,17.0,204.0,1448,-4.0,425.882,7888,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,25.0,210.0,1448,-14.0,413.714,7888,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,11.0,215.0,1542,-4.0,430.326,7888,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
4,21.0,323.0,2552,-9.0,474.056,7888,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818548,9.0,72.0,196,6.0,163.333,12,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2818549,5.0,72.0,196,6.0,163.333,12,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2818550,16.0,72.0,196,-1.0,163.333,12,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2818551,6.0,72.0,196,-4.0,163.333,12,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


## Neural Networks

In [11]:
NN_X = LR_train.drop(columns=['ARRIVAL_DELAY']).loc[:10000,:]
NN_Y = LR_train.loc[:10000,"ARRIVAL_DELAY"]

NN_X = pd.get_dummies(NN_X, columns=['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT','CITY','STATE','area'], prefix = ['Airline','Origin','Dest','city','state','area'])
NN_X

,TAXI_OUT,SCHEDULED_TIME,DISTANCE,DEPARTURE_DELAY,SPEED,Count Flights,Airline_AA,Airline_AS,Airline_B6,Airline_DL,...,Dest_SMF,Dest_SMX,Dest_STL,Dest_TPA,Dest_TUS,city_Anchorage,city_Los Angeles,state_AK,state_CA,area_west
0,21.0,205.0,1448,-11.0,423.805,7888,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
1,17.0,204.0,1448,-4.0,425.882,7888,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
2,25.0,210.0,1448,-14.0,413.714,7888,0,0,0,1,...,0,0,0,0,0,1,0,1,0,1
3,11.0,215.0,1542,-4.0,430.326,7888,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
4,21.0,323.0,2552,-9.0,474.056,7888,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,18.0,245.0,1744,51.0,427.102,103511,1,0,0,0,...,0,0,0,0,0,0,1,0,1,1
9997,19.0,77.0,308,13.0,240.000,103511,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
9998,15.0,250.0,1947,4.0,467.280,103511,0,0,0,1,...,0,0,0,0,0,0,1,0,1,1
9999,15.0,60.0,209,38.0,209.000,103511,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1


In [12]:
# define base model
def baseline_model():
    
    # create model
    model = Sequential()
    
    # ensure the input layer has the right number of input features - input_dim argument
    # specify the number of neurons or nodes in the layer as the first argument
    # specify the activation function using the activation argument
    model.add(Dense(13, input_dim=118, kernel_initializer='normal', activation='relu'))
    
    # No activation function is used for the output layer because it is a regression problem and we 
    # are interested in predicting numerical values directly without transform
    model.add(Dense(1, kernel_initializer='normal'))
    
    # Compile model - specify the loss function to use to evaluate a set of weights
    # the optimizer is used to search through different weights for the network and any optional metrics 
    # we would like to collect and report during training
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

Adam optimizer - efficient stochastic gradient descent algorithm. This is a popular version of gradient descent because it automatically tunes itself and gives good results in a wide range of problems.

More info:
https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/

It used to be the case that Sigmoid and Tanh activation functions were preferred for all layers. These days, better performance is achieved using the ReLU activation function.

We could also try other activation functions here (?)

#### Fitting Keras Model

Training occurs over epochs and each epoch is split into batches.
* Epoch: One pass through all of the rows in the training dataset.
* Batch: One or more samples considered by the model within an epoch before weights are updated.

These configurations can be chosen experimentally by trial and error.

In [32]:
# fit the keras model on the dataset - we definitely need to run it with much more epochs
NN_model = baseline_model()
NN_model.fit(NN_X, NN_Y, epochs=15, batch_size=10)

Epoch 1/15
1001/1001 [==============================] - 2s 1ms/step - loss: 1080.6431
Epoch 2/15
1001/1001 [==============================] - 1s 1ms/step - loss: 1021.6476
Epoch 3/15
1001/1001 [==============================] - 1s 1ms/step - loss: 950.4149
Epoch 4/15
1001/1001 [==============================] - 1s 1ms/step - loss: 915.8968
Epoch 5/15
1001/1001 [==============================] - 1s 1ms/step - loss: 821.7077
Epoch 6/15
1001/1001 [==============================] - 1s 1ms/step - loss: 702.7849
Epoch 7/15
1001/1001 [==============================] - 1s 1ms/step - loss: 597.4328
Epoch 8/15
1001/1001 [==============================] - 1s 1ms/step - loss: 514.7346
Epoch 9/15
1001/1001 [==============================] - 1s 1ms/step - loss: 449.4120
Epoch 10/15
1001/1001 [==============================] - 1s 1ms/step - loss: 386.7083
Epoch 11/15
1001/1001 [==============================] - 1s 1ms/step - loss: 344.2870
Epoch 12/15
1001/1001 [==============================] - 2s 2

### Evaluaion of Keras Model
10-fold cross validation to evaluate the model.

In [91]:
#!pip install scikeras

In [33]:
import scikeras
from scikeras.wrappers import KerasRegressor

estimator = KerasRegressor(model=baseline_model, epochs=10, batch_size=5, verbose=0)
kfold = KFold(n_splits=3)
results = cross_val_score(estimator, NN_X, NN_Y, cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Results: -1.05 (2.50) MSE


In [33]:
# is MSE  normalized here? I'm not sure what this value means

### Predicting on train set

In [34]:
# make probability predictions with the model
predictions = NN_model.predict(NN_X)
# round predictions 
#rounded = [round(x[0]) for x in predictions]
print(predictions)

[[-12.301533 ]
 [ -7.0537763]
 [-13.939857 ]
 ...
 [-16.710426 ]
 [ 23.639229 ]
 [175.18524  ]]


In [35]:
print("Mean squared error - Train: %.2f" % mean_squared_error(NN_Y, predictions))
print('Variance score - Train: %.2f' % r2_score(NN_Y, predictions))

Mean squared error - Train: 289.75
Variance score - Train: 0.73


## Training on the whole dataset

In [13]:
net_X = LR_train.drop(columns=['ARRIVAL_DELAY','CITY','STATE','area','Count Flights'])
net_Y = LR_train["ARRIVAL_DELAY"]

net_X = pd.get_dummies(net_X, columns=['AIRLINE','ORIGIN_AIRPORT','DESTINATION_AIRPORT'], prefix = ['Airline','Origin','Dest'])
net_X

,TAXI_OUT,SCHEDULED_TIME,DISTANCE,DEPARTURE_DELAY,SPEED,Airline_AA,Airline_AS,Airline_B6,Airline_DL,Airline_EV,...,Dest_TYS,Dest_UST,Dest_VEL,Dest_VLD,Dest_VPS,Dest_WRG,Dest_WYS,Dest_XNA,Dest_YAK,Dest_YUM
0,21.0,205.0,1448,-11.0,423.805,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,17.0,204.0,1448,-4.0,425.882,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,25.0,210.0,1448,-14.0,413.714,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,11.0,215.0,1542,-4.0,430.326,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,21.0,323.0,2552,-9.0,474.056,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818548,9.0,72.0,196,6.0,163.333,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2818549,5.0,72.0,196,6.0,163.333,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2818550,16.0,72.0,196,-1.0,163.333,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2818551,6.0,72.0,196,-4.0,163.333,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
def net_model():
    model = Sequential()
    
    # input_dim = number of columns from DataFrame above
    model.add(Dense(13, input_dim=663, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam')
    
    return model

In [59]:
# Model trained on the whole dataset, this took a while

NN_model = net_model()
NN_model.fit(net_X, net_Y, epochs=15, batch_size=10)

Epoch 1/15
281856/281856 [==============================] - 648s 2ms/step - loss: 183.8735
Epoch 2/15
281856/281856 [==============================] - 462s 2ms/step - loss: 150.2612
Epoch 3/15
281856/281856 [==============================] - 469s 2ms/step - loss: 136.8132
Epoch 4/15
281856/281856 [==============================] - 458s 2ms/step - loss: 132.7287
Epoch 5/15
281856/281856 [==============================] - 433s 2ms/step - loss: 130.9817
Epoch 6/15
281856/281856 [==============================] - 417s 1ms/step - loss: 129.9200
Epoch 7/15
281856/281856 [==============================] - 502s 2ms/step - loss: 128.2380
Epoch 8/15
281856/281856 [==============================] - 496s 2ms/step - loss: 128.3165
Epoch 9/15
281856/281856 [==============================] - 476s 2ms/step - loss: 128.9166
Epoch 10/15
281856/281856 [==============================] - 474s 2ms/step - loss: 128.8585
Epoch 11/15
281856/281856 [==============================] - 500s 2ms/step - loss: 127.79

In [60]:
predictions = NN_model.predict(net_X)
print(predictions)
print("Mean squared error - Train: %.2f" % mean_squared_error(net_Y, predictions))
print('Variance score - Train: %.2f' % r2_score(net_Y, predictions))

[[ -5.2197113]
 [ -2.0366554]
 [ -9.020018 ]
 ...
 [ -4.3686404]
 [-18.092918 ]
 [-11.574679 ]]
Mean squared error - Train: 122.55
Variance score - Train: 0.93


In [64]:
#!pip install h5py

### Saving model to JSON

In [65]:
# serialize model to JSON
# separately saving wiights and architecture
model_json = NN_model.to_json()
with open("NN_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
NN_model.save_weights("NN_model.h5")
print("Saved NN_model to disk")

Saved NN_model to disk


In [68]:
# LOADING model from JSON
from keras.models import model_from_json

json_file = open('NN_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("NN_model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='mean_squared_error', optimizer='adam')
# score = loaded_model.evaluate(net_X, net_Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

predictions_loaded = loaded_model.predict(net_X)
print(predictions_loaded)
print("Mean squared error - Train: %.2f" % mean_squared_error(net_Y, predictions_loaded))
print('Variance score - Train: %.2f' % r2_score(net_Y, predictions_loaded))

Loaded model from disk
[[ -5.2197113]
 [ -2.0366554]
 [ -9.020018 ]
 ...
 [ -4.3686404]
 [-18.092918 ]
 [-11.574679 ]]
Mean squared error - Train: 122.55
Variance score - Train: 0.93


### Save Model Weights and Architecture Together - no need to compile again

In [14]:
# save model and architecture to single file
NN_model.save("model.h5")
print("Saved NN_model to disk")

# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model

# load model
new_model = load_model('model.h5')
# summarize model.
new_model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 13)                8632      
                                                                 
 dense_21 (Dense)            (None, 1)                 14        
                                                                 
Total params: 8,646
Trainable params: 8,646
Non-trainable params: 0
_________________________________________________________________


In [15]:
new_predictions_loaded = new_model.predict(net_X)
print(new_predictions_loaded)
print("Mean squared error - Train: %.2f" % mean_squared_error(net_Y, new_predictions_loaded))
print('Variance score - Train: %.2f' % r2_score(net_Y, new_predictions_loaded))

[[ -5.2197113]
 [ -2.0366554]
 [ -9.020018 ]
 ...
 [ -4.3686404]
 [-18.092918 ]
 [-11.574679 ]]
Mean squared error - Train: 122.55
Variance score - Train: 0.93
